# PCAI PDF RAG System

## Complete Implementation of 5 Required Components

This notebook demonstrates a working Retrieval-Augmented Generation system using PDF File

**Components Implemented:**
1. Knowledge Base (PDF)
2. Semantic Layer (sentence-transformers)
3. Retrieval System (FAISS)
4. Augmentation (context enrichment)
5. Generation (answer synthesis)

## Setup: Import All Components

Run this cell first to load all modules.

In [1]:
# Import our 5 components
from knowledge_base import KnowledgeBase
from semantic_layer import SemanticLayer
from retrieval_system import RetrievalSystem
from augmentation import Augmentation
from generation import Generation
print("✅ All components imported successfully!")

✅ All components imported successfully!


/Users/nallagat/playground/at-2/rag-venv/lib/python3.12/site-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(


## Component 1: Knowledge Base

Load PCAI dataset from pdf.

In [2]:

# Initialize KB with PDF and chunk settings
kb = KnowledgeBase(pdf_path="data/hpe-pcai.pdf", chunk_size=200, chunk_overlap=50)

# Load PDF pages
documents = kb.load_pdf_data()

# Create sentence-preserving chunks
chunks = kb.create_chunks()

# Inspect stats
stats = kb.get_stats()
stats = kb.get_stats()
print(f"\n📊 Knowledge Base:")
print(f"   Pages: {stats['total_pages']}")
print(f"   Chunks: {stats['total_chunks']}")
print(f"   Source: {stats['source']}")


📚 Knowledge Base initialized

📥 Loading PDF: data/hpe-pcai.pdf
✅ Loaded 24 pages from PDF
✅ Created 57 text chunks

📊 Knowledge Base:
   Pages: 24
   Chunks: 57
   Source: data/hpe-pcai.pdf


## Component 2: Semantic Layer

Convert text to numerical embeddings for semantic comparison.

In [ ]:
# Initialize Semantic Layer
semantic = SemanticLayer()

# Encode all documents to embeddings
document_embeddings = semantic.encode_documents(documents)

print(f"\n📊 Embedding Statistics:")
print(f"   Shape: {document_embeddings.shape}")
print(f"   Meaning: {document_embeddings.shape[0]} documents")
print(f"   Each represented by {document_embeddings.shape[1]} numbers")

In [3]:
import random
import numpy as np

# =========================
# Random Preview: Chunks & Embeddings
# =========================

# Number of random samples to preview
N = 5

# Ensure embeddings exist
try:
    document_embeddings
except NameError:
    # Encode chunks if not done yet
    semantic = SemanticLayer()
    document_embeddings = semantic.encode_documents(kb.chunks)

# Random indices
random_indices = random.sample(range(len(kb.chunks)), min(N, len(kb.chunks)))

print(f"\n📄 Random preview of {N} chunks with embeddings:\n")

for i, idx in enumerate(random_indices):
    chunk = kb.chunks[idx]
    emb = document_embeddings[idx]
    
    print(f"--- Sample {i+1} (Index {idx}) ---")
    print(f"Page: {chunk['page']}")
    print(f"Text (first 200 chars): {chunk['chunk'][:200]}...")
    print(f"Embedding shape: {emb.shape}")
    print(f"Embedding sample (first 10 values): {emb[:10]}")
    print("-" * 70)


🧠 COMPONENT 2: Semantic Layer initializing...
   Loading model: all-MiniLM-L6-v2
   (First run downloads 22MB - takes 30 seconds)
✅ Model loaded! Embeddings dimension: 384

🔄 Encoding 57 document chunks to embeddings...


/Users/nallagat/playground/at-2/rag-venv/lib/python3.12/site-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(


Batches:   0%|          | 0/2 [00:00<?, ?it/s]

✅ Embeddings created: (57, 384)

📄 Random preview of 5 chunks with embeddings:

--- Sample 1 (Index 32) ---
Page: 12
Text (first 200 chars): supply branch circuits The total system AC load does not exceed 80% of the branch circuit's rated AC For installations using a UPS, the load must not exceed 80% of the UPS's marked electrical current ...
Embedding shape: (384,)
Embedding sample (first 10 values): [ 0.01328968  0.03771245 -0.07328703 -0.05400196 -0.03626148  0.0047138
  0.00410422  0.09722557  0.02806317  0.05871234]
----------------------------------------------------------------------
--- Sample 2 (Index 36) ---
Page: 15
Text (first 200 chars): Network bandwidth and software updates Network bandwidth and software updates HPE Private Cloud AI requires a sustained Internet connection of at least 100 Mbps to ensure that software updates finish ...
Embedding shape: (384,)
Embedding sample (first 10 values): [-0.01714158 -0.03546501  0.05789873 -0.08162022  0.03032633 -0.01002431
 -0.

## Component 3: Retrieval System

Build FAISS index for fast similarity search.

In [4]:
# Initialize Retrieval System
retrieval = RetrievalSystem(dimension=384)

# Build searchable index
retrieval.build_index(document_embeddings, documents)

print("\n✅ Retrieval system ready to search!")

🔍 COMPONENT 3: Retrieval System initialized (dim=384)

🔨 Building FAISS index...
   Documents: 24
   Embedding shape: (57, 384)
✅ Index built with 57 vectors

✅ Retrieval system ready to search!


In [7]:
# Inspect retrieval output
results = retrieval.search(semantic.encode_query("What are pcai core components"), top_k=3)
print("Results structure:\n")
for i, r in enumerate(results):
    print(f"Result {i}:")
    print(r)
    print("-" * 50)



🔎 Searching for top 3 relevant documents...
✅ Retrieved 3 documents
Results structure:

Result 0:
{'rank': 1, 'document': {'page': 1, 'content': 'HPE Private Cloud AI 1.5 Administration Guide\nHPE Private Cloud AI 1.5 Administration Guide\nPart Number: 20-PCAI-UG-ED1\nPublished: September 2025\nEdition: 1.5'}, 'similarity': 0.4387816354426174, 'distance': 1.2790379524230957}
--------------------------------------------------
Result 1:
{'rank': 2, 'document': {'page': 5, 'content': "Scalability\nScalability\n: \nHPE\n \nPrivate Cloud AI\n's modular design allows organizations to start small and expand their AI capabilities as needed,\nwithout major disruptions or reinvestments.\nSimplified management\nSimplified management\n: \nHPE\n AI Essentials\n provides comprehensive management tools and automation features, reducing the\noperational complexity of running AI infrastructure.\nEcosystem integration\nEcosystem integration\n: \nHPE\n \nPrivate Cloud AI\n is designed to work seamlessly

## Test the Complete RAG Pipeline

Now let's ask a question and see all 5 components work together!

In [8]:
def ask_question(question, top_k=3):
    """Complete RAG pipeline for a question using PCAI PDF knowledge base"""
    
    print("=" * 70)
    print(f"❓ QUESTION: {question}")
    print("=" * 70)
    
    # COMPONENT 2: Encode query
    print("\n[Step 1] Encoding question...")
    query_embedding = semantic.encode_query(question)
    
    # COMPONENT 3: Retrieve relevant chunks
    print("[Step 2] Searching knowledge base...")
    results = retrieval.search(query_embedding, top_k=top_k)
    
    print(f"\n📚 Retrieved Top {top_k} Chunks:")
    for r in results:
        doc_page = r['document']['page']
        doc_text = r['document']['content']
        print(f"  • Page {doc_page} (similarity: {r['similarity']:.1%})")
        print(f"    Text snippet: {doc_text[:200]}...\n")
    
    # COMPONENT 4: Augment with context
    print("[Step 3] Creating enriched context...")
    augmentor = Augmentation()
    enriched = augmentor.create_context(question, results)
    
    # COMPONENT 5: Generate answer
    print("[Step 4] Generating answer...")
    generator = Generation()
    answer = generator.generate(enriched, results)
    
    # Display answer
    print("\n" + "=" * 70)
    print("💬 ANSWER:")
    print("=" * 70)
    print(answer)
    print("=" * 70)
    
    return answer

# Example usage
ask_question("What are pcai core tools")



❓ QUESTION: What are pcai core tools

[Step 1] Encoding question...
[Step 2] Searching knowledge base...

🔎 Searching for top 3 relevant documents...
✅ Retrieved 3 documents

📚 Retrieved Top 3 Chunks:
  • Page 16 (similarity: 47.9%)
    Text snippet: Red Hat Enterprise Linux (RHEL)
Red Hat Enterprise Linux (RHEL)
HPE
 sends an email receipt with the order
details and instructions to register the
products. The email is sent to the contact on
the Pu...

  • Page 20 (similarity: 47.7%)
    Text snippet: Virtualization
Virtualization
VM Essentials
Data Service Connectors
Data Service Connectors
HPE
-DSC VM with equivalent KickStart Scripts
Data Engineering Tools
Data Engineering Tools
Apache Airflow
A...

  • Page 21 (similarity: 47.3%)
    Text snippet: HPE Private Cloud AI 1.5 Administration Guide
21...

[Step 3] Creating enriched context...
⚡ COMPONENT 4: Augmentation initialized

⚡ Creating enriched context with 3 documents...
✅ Enriched context created (3858 characters)
[Step 4] Gener

'Based on 3 relevant scenes from Friends:\n\n1. Page 16 (relevance: 47.9%)\n   Red Hat Enterprise Linux (RHEL)\nRed Hat Enterprise Linux (RHEL)\nHPE\n sends an email receipt with the order\ndetails and instructions to register the\nproducts. The email is sent to the contact on\nthe Pu...\n\n2. Page 20 (relevance: 47.7%)\n   Virtualization\nVirtualization\nVM Essentials\nData Service Connectors\nData Service Connectors\nHPE\n-DSC VM with equivalent KickStart Scripts\nData Engineering Tools\nData Engineering Tools\nApache Airflow\nA...\n\n3. Page 21 (relevance: 47.3%)\n   HPE Private Cloud AI 1.5 Administration Guide\n21\n'

## Try More Questions!

Test the system with different queries.

In [9]:
# Try these questions or write your own!

ask_question("What is pcai engineered system for?")

❓ QUESTION: What is pcai engineered system for?

[Step 1] Encoding question...
[Step 2] Searching knowledge base...

🔎 Searching for top 3 relevant documents...
✅ Retrieved 3 documents

📚 Retrieved Top 3 Chunks:
  • Page 21 (similarity: 46.9%)
    Text snippet: HPE Private Cloud AI 1.5 Administration Guide
21...

  • Page 1 (similarity: 45.6%)
    Text snippet: HPE Private Cloud AI 1.5 Administration Guide
HPE Private Cloud AI 1.5 Administration Guide
Part Number: 20-PCAI-UG-ED1
Published: September 2025
Edition: 1.5...

  • Page 5 (similarity: 45.4%)
    Text snippet: Scalability
Scalability
: 
HPE
 
Private Cloud AI
's modular design allows organizations to start small and expand their AI capabilities as needed,
without major disruptions or reinvestments.
Simplifi...

[Step 3] Creating enriched context...
⚡ COMPONENT 4: Augmentation initialized

⚡ Creating enriched context with 3 documents...
✅ Enriched context created (2904 characters)
[Step 4] Generating answer...
🤖 COMPONENT 5: Ge

"Based on 3 relevant scenes from Friends:\n\n1. Page 21 (relevance: 46.9%)\n   HPE Private Cloud AI 1.5 Administration Guide\n21\n\n2. Page 1 (relevance: 45.6%)\n   HPE Private Cloud AI 1.5 Administration Guide\nHPE Private Cloud AI 1.5 Administration Guide\nPart Number: 20-PCAI-UG-ED1\nPublished: September 2025\nEdition: 1.5\n\n3. Page 5 (relevance: 45.4%)\n   Scalability\nScalability\n: \nHPE\n \nPrivate Cloud AI\n's modular design allows organizations to start small and expand their AI capabilities as needed,\nwithout major disruptions or reinvestments.\nSimplifi...\n"

In [10]:
ask_question("is spark used with livy")

❓ QUESTION: is spark used with livy

[Step 1] Encoding question...
[Step 2] Searching knowledge base...

🔎 Searching for top 3 relevant documents...


IndexError: list index out of range

In [11]:
# Your own question here!
ask_question("What is the hardware used")

❓ QUESTION: What is the hardware used

[Step 1] Encoding question...
[Step 2] Searching knowledge base...

🔎 Searching for top 3 relevant documents...


IndexError: list index out of range

## Interactive Mode

Keep asking questions!

In [ ]:
# Interactive questioning
while True:
    user_question = input("\nYour question (or 'quit' to stop): ").strip()
    
    if user_question.lower() in ['quit', 'exit', 'q']:
        print("👋 PDF RAG here to answer")
        break
    
    if user_question:
        ask_question(user_question)
    else:
        print("Please enter a question!")